# Imports

In [ ]:
import numpy as np
from scipy.io import loadmat, whosmat, savemat
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import os
import re
import pandas as pd
from tqdm import tqdm
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import pickle
import gc
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, cross_val_score
from sklearn.utils import shuffle
from sklearn.neural_network import MLPClassifier
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Utils

In [ ]:
base_path = ''

In [ ]:
# extract
def ends_with_specific_number(input_string, specific_number):
    pattern = re.compile(rf".*{specific_number}$")
    return bool(pattern.match(input_string))
'''
Sorts files based on:
  1. subject (Asc)
  2. experiment date (Asc)
'''

def custom_sort(file_name):
    # Extract the number before the underscore and the date
    match = re.match(r'(\d+)_(\d+)', file_name)
    if match:
        number_part = int(match.group(1))
        date_part = int(match.group(2))
        return (number_part, date_part)
    else:
        return (float('inf'), float('inf'))

# Loading Data

In [ ]:
data = np.zeros((675,62,4000))

In [ ]:
labels = loadmat(os.path.join(base_path, 'Preprocessed_EEG/label.mat'))['label'][0]
labels_edited = np.empty(675)
for i in range(0,45):
  labels_edited[i*15:(i+1)*15] = labels

In [ ]:
p = os.path.join(base_path, 'Preprocessed_EEG')
for j, file in tqdm(enumerate(sorted([f for f in os.listdir(p) if '.mat' in f and '_' in f], key=custom_sort))):
  # print(j,file)
  mat = loadmat(os.path.join(p, file))
  for i, k in enumerate(list(mat.keys())[3:]):
    data[j*15+i,:,:] = mat[k][:,-4001:-1]

45it [04:10,  5.56s/it]


In [ ]:
data = data.reshape(675,-1)

# Raw Data - 5 Fold - Last 20 Seconds

In [ ]:
results_dict ={
  "SVM": [],
  "K-Nearest Neighbors":[],
  "Logistic Regression": []
}
kf = KFold(n_splits=5, shuffle=True, random_state=42)
for train_index, test_index in kf.split(data):
  train_data, test_data = data[train_index], data[test_index]
  train_labels, test_labels = labels_edited[train_index], labels_edited[test_index]
  classifiers = {
  "SVM": SVC(),
  "K-Nearest Neighbors": KNeighborsClassifier(),
  "Logistic Regression": LogisticRegression()
  }

  for name, clf in classifiers.items():
    clf.fit(train_data, train_labels)
    predictions = clf.predict(test_data)
    accuracy = accuracy_score(test_labels, predictions)
    print(f"{name} Accuracy: {accuracy}")
    results_dict[name].append(accuracy)

df = pd.DataFrame(results_dict)
df.to_csv('rawdata-5fold-last20sec.csv')

SVM Accuracy: 0.2962962962962963
K-Nearest Neighbors Accuracy: 0.35555555555555557
Logistic Regression Accuracy: 0.6444444444444445
SVM Accuracy: 0.2814814814814815
K-Nearest Neighbors Accuracy: 0.37037037037037035
Logistic Regression Accuracy: 0.45925925925925926
SVM Accuracy: 0.28888888888888886
K-Nearest Neighbors Accuracy: 0.3333333333333333
Logistic Regression Accuracy: 0.5555555555555556
SVM Accuracy: 0.2814814814814815
K-Nearest Neighbors Accuracy: 0.32592592592592595
Logistic Regression Accuracy: 0.5111111111111111
SVM Accuracy: 0.28888888888888886
K-Nearest Neighbors Accuracy: 0.35555555555555557
Logistic Regression Accuracy: 0.5333333333333333
